# Azure Sanal Makine CPU Kullanım Analizi

In [1]:
import pandas as pd
import plotly.express as px
import os
import numpy as np

## Adım 1: Veri Dosyalarını Yükleme ve Birleştirme

Bu adımda, `data` klasörüne indirdiğimiz 3 adet `vm_cpu_readings` dosyasını okuyup tek bir `DataFrame` altında birleştireceğiz.

In [2]:
# Konteyner içindeki veri klasörünün yolu
data_path = '/app/data/'

# https://github.com/Azure/AzurePublicDataset
files_to_read = [f"vm_cpu_readings-file-{i}-of-195.csv.gz" for i in range(1, 11)]

# Schema'ya göre sütun isimleri
column_names = ['timestamp', 'vmId', 'min_cpu', 'max_cpu', 'avg_cpu']

df_list = []

print("Dosyalar okunuyor...")
for file_name in files_to_read:
    full_path = os.path.join(data_path, file_name)
    
    if os.path.exists(full_path):
        temp_df = pd.read_csv(
            full_path,
            header=None,
            names=column_names,
            compression='gzip'
        )
        df_list.append(temp_df)
        print(f"- '{file_name}' okundu.")
    else:
        print(f"- UYARI: '{file_name}' bulunamadı.")

if df_list:
    df = pd.concat(df_list, ignore_index=True)
    print("\nTüm veriler başarıyla birleştirildi!")
else:
    print("\nOkunacak veri dosyası bulunamadı.")

Dosyalar okunuyor...
- 'vm_cpu_readings-file-1-of-195.csv.gz' okundu.
- 'vm_cpu_readings-file-2-of-195.csv.gz' okundu.
- 'vm_cpu_readings-file-3-of-195.csv.gz' okundu.
- 'vm_cpu_readings-file-4-of-195.csv.gz' okundu.
- 'vm_cpu_readings-file-5-of-195.csv.gz' okundu.
- 'vm_cpu_readings-file-6-of-195.csv.gz' okundu.
- 'vm_cpu_readings-file-7-of-195.csv.gz' okundu.
- 'vm_cpu_readings-file-8-of-195.csv.gz' okundu.
- 'vm_cpu_readings-file-9-of-195.csv.gz' okundu.
- 'vm_cpu_readings-file-10-of-195.csv.gz' okundu.

Tüm veriler başarıyla birleştirildi!


### Verinin Genel Yapısını İnceleme

In [3]:
if 'df' in locals():
    print("Verinin ilk 5 satırı:")
    display(df.head())
    print("\nVeri hakkında genel bilgiler:")
    df.info()

Verinin ilk 5 satırı:


,timestamp,vmId,min_cpu,max_cpu,avg_cpu
0,0,yNf/R3X8fyXkOJm3ihXQcT0F52a8cDWPPRzTT6QFW8N+1Q...,19.898441,24.996361,22.630673
1,0,gVb4X4iS13nJrM0KZsy7SrHzWAHix0CEPlK7/deV5vkwjt...,14.281488,20.028324,17.250655
2,0,f8BtQHczrXFjnVxWl8Hqm1kH9UD/8nCvtXCLiFvVRvamqa...,1.655368,23.138685,8.533818
3,0,8xjtVrJRJAyArNlbRgCftoNQOZiWl2eRP6uQavL9+6IULT...,47.501061,52.940579,50.045584
4,0,zTE3f0H2n43tW+PA3OdonjUTWWxeyzF7xJk9QH9s/487J/...,1.724155,12.889580,4.833578



Veri hakkında genel bilgiler:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000000 entries, 0 to 99999999
Data columns (total 5 columns):
 #   Column     Dtype  
---  ------     -----  
 0   timestamp  int64  
 1   vmId       object 
 2   min_cpu    float64
 3   max_cpu    float64
 4   avg_cpu    float64
dtypes: float64(3), int64(1), object(1)
memory usage: 3.7+ GB


## Adım 2: Analiz İçin Tek Bir Sanal Makine (VM) Seçme

Veri setindeki binlerce makine arasından, üzerinde en çok kayıt bulunanı seçerek analizimizi basitleştireceğiz.

In [4]:
if 'df' in locals():
    # En çok veri kaydına sahip olan VM
    target_vm_id = df['vmId'].value_counts().idxmax()
    print(f"Analiz için en çok veriye sahip olan '{target_vm_id}' ID'li VM seçildi.")

    # Sadece bu VM'e ait verileri filtrele
    df_single_vm = df[df['vmId'] == target_vm_id].copy()

Analiz için en çok veriye sahip olan 'ef/Uph6sHGxOk9osPZNARutg0JeyAYagnPfsSr8TfYVf69sBLXydFO78zJLGpF/n' ID'li VM seçildi.


## Adım 3: Zaman Serisi Hazırlığı ve Görselleştirme

Seçtiğimiz tek VM'in verisini zaman serisi analizine uygun hale getirip grafiğini çizeceğiz.

In [5]:
if 'df_single_vm' in locals():
    # Timestamp sütununu okunabilir tarih formatına çevirelim
    # Azure veri seti saniye cinsinden zaman damgası kullanır (unit='s')
    df_single_vm['datetime'] = pd.to_datetime(df_single_vm['timestamp'], unit='s')

    # Tarih/zaman sütununu index olarak atayalım, bu zaman serisi analizleri için çok kullanışlıdır
    df_single_vm.set_index('datetime', inplace=True)
    df_single_vm.sort_index(inplace=True)

    print(f"'{target_vm_id}' için hazırlanmış verinin ilk 5 satırı:")
    display(df_single_vm.head())

'ef/Uph6sHGxOk9osPZNARutg0JeyAYagnPfsSr8TfYVf69sBLXydFO78zJLGpF/n' için hazırlanmış verinin ilk 5 satırı:


,timestamp,vmId,min_cpu,max_cpu,avg_cpu
datetime,,,,,
1970-01-01 00:00:00,0,ef/Uph6sHGxOk9osPZNARutg0JeyAYagnPfsSr8TfYVf69...,0.792217,3.863575,1.189029
1970-01-01 00:05:00,300,ef/Uph6sHGxOk9osPZNARutg0JeyAYagnPfsSr8TfYVf69...,0.769778,0.947936,0.856719
1970-01-01 00:10:00,600,ef/Uph6sHGxOk9osPZNARutg0JeyAYagnPfsSr8TfYVf69...,0.772892,4.667597,1.525741
1970-01-01 00:15:00,900,ef/Uph6sHGxOk9osPZNARutg0JeyAYagnPfsSr8TfYVf69...,0.747099,0.922060,0.846243
1970-01-01 00:20:00,1200,ef/Uph6sHGxOk9osPZNARutg0JeyAYagnPfsSr8TfYVf69...,0.707030,0.978423,0.846580


In [6]:
display(df_single_vm.tail())

,timestamp,vmId,min_cpu,max_cpu,avg_cpu
datetime,,,,,
1970-01-02 12:45:00,132300,ef/Uph6sHGxOk9osPZNARutg0JeyAYagnPfsSr8TfYVf69...,0.719798,0.951096,0.844752
1970-01-02 12:50:00,132600,ef/Uph6sHGxOk9osPZNARutg0JeyAYagnPfsSr8TfYVf69...,0.699789,0.940443,0.841434
1970-01-02 12:55:00,132900,ef/Uph6sHGxOk9osPZNARutg0JeyAYagnPfsSr8TfYVf69...,0.701524,0.996953,0.853858
1970-01-02 13:00:00,133200,ef/Uph6sHGxOk9osPZNARutg0JeyAYagnPfsSr8TfYVf69...,0.734421,0.971547,0.856805
1970-01-02 13:05:00,133500,ef/Uph6sHGxOk9osPZNARutg0JeyAYagnPfsSr8TfYVf69...,0.710948,4.072359,1.196745


In [7]:
if 'df_single_vm' in locals():
    print("Grafik oluşturuluyor...")
    fig = px.line(
        df_single_vm,
        y='avg_cpu',
        title=f"Sanal Makine (vmId: {target_vm_id}) Ortalama CPU Kullanımı (4 Dosyadan)"
    )
    fig.show()

Grafik oluşturuluyor...


In [8]:
unique_vms = df['vmId'].unique()

sample_size = min(5, len(unique_vms))
random_sample_vms = np.random.choice(unique_vms, size=sample_size, replace=False)

print(f"Grafik için rastgele seçilen VM'ler: {random_sample_vms}")

# Sadece bu rastgele seçilen VM'lere ait verileri filtreleyelim
df_sample = df[df['vmId'].isin(random_sample_vms)].copy()

# Zaman damgasını okunabilir formata çevirelim
df_sample['datetime'] = pd.to_datetime(df_sample['timestamp'], unit='s')

# Grafiği çizdirelim. 'color' parametresi her bir VM'e farklı bir renk atayacak.
fig = px.line(
    df_sample,
    x='datetime',
    y='avg_cpu',
    color='vmId', # Bu satır her VM için ayrı bir çizgi oluşturur
    title='Rastgele Seçilmiş 5 Sanal Makinenin Ortalama CPU Kullanımı'
)
fig.show()

Grafik için rastgele seçilen VM'ler: ['4yvDC+ot1w3RVcHHYzeLNtRGrslo2/rAmQvVZm0xXRkgnGBx9UGTUmTrAHcv0N+U'
 'NkDpmG5S8l9nCkvYAd0aQ60qYP84vCcteybc+y7Xtb46uxPDw2D8TsP8MegmG3C6'
 'v0tL0YbIiEXmud3qUMy+BovRmy2sv57rhNkcjBwgDI5M5FOKNZJ9bNutr68HHbhK'
 '+rzWv8Md2vsW8hOroJPpd0AEYjXJ341kQV9rhNiM34idnr2MwV7V+AGju4j/7xdF'
 'pC9dLxk5xjCBg0cP4PNSv4daqLYVEs7tuOPzhx4WMDPo13r3kSzt7Us0FpsHQXka']
